### Importing required libraries and reading the file

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import re
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import cdist
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import normalize
from wordcloud import WordCloud
from datetime import datetime
import random
from tqdm import tqdm
from gensim.models import Word2Vec

In [53]:
tlog = pd.read_csv(r"C:\Users\rtd91\Data\mlops\data\sample_tlog.csv")
product = pd.read_csv(r"C:\Users\rtd91\Data\mlops\data\sample_product.csv")
customer = pd.read_csv(r"C:\Users\rtd91\Data\mlops\data\sample_customer.csv")

In [54]:
def preprocessing(x):  
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english')) 
    filtered_tokenized_text=[]
    tokenized_text = nltk.sent_tokenize(x)
    for sentence in tokenized_text:
        sentence = sentence.lower()
        word_tokens = word_tokenize(sentence)
        filtered_sentence = [ps.stem(w) for w in word_tokens if not w in stop_words]
        filtered_tokenized_text.append(filtered_sentence)
    return "".join([' '.join(text) for text in filtered_tokenized_text])

In [55]:
print(len(tlog["transaction_id"]),tlog["transaction_id"].nunique())
print(len(tlog["customer_id"]),tlog["customer_id"].nunique())
print(len(tlog["upc_no"]),tlog["upc_no"].nunique())

595306 86108
595306 1243
595306 2938


In [56]:
print(len(product["upc_no"]),product["upc_no"].nunique())

2938 2938


In [6]:
print(len(customer["customer_id"]),customer["customer_id"].nunique())

1243 1243


In [9]:
product.head()

,Unnamed: 0,upc_no,upc_desc,category_no_level_1,category_desc_level_1,category_no_level_2,category_desc_level_2
0,0,4600028876,OEP MLD TACO SEASONING MX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS
1,1,5210003491,MCCO ORIG TACO SEASNG MIX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS
2,2,2100003934,TACOBL ORIG TACO SSNG MIX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS
3,3,1111071507,KRO FAJITA SEASONING MIX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS
4,4,1111071503,KRO REG TACO SEASONING MX,2053,MEXICAN SEASONING MIXES,93,TRADITIONAL MEXICAN FOODS


In [10]:
customer.head()

,Unnamed: 0,customer_id,convenience_dim_seg,quality_dim_seg,health_dim_seg,price_dim_seg,convenience_dim_score,quality_dim_score,health_dim_score,price_dim_score
0,0,60622933,M,M,M,M,57.237420,58.455336,46.045199,56.256278
1,1,116318566,M,L,M,M,55.043287,36.847263,49.923244,57.525922
2,2,290772182,M,M,M,H,51.416044,46.398561,48.046043,66.436402
3,3,97514036,M,M,M,M,53.418191,57.848860,51.320066,62.298526
4,4,346208265,H,M,L,M,59.661310,57.548050,48.700522,62.308051


In [11]:
tlog.head()

,Unnamed: 0,transaction_id,customer_id,upc_no,category_no_level_1,trans_dt,week_no,store_id,region_cd,purchase_unit,sales_amt,disc_amt,purchase_price
0,0,014_00817_4_121_20190313_18:04:00,14595593,94011,62002,20190313,20190202,817,14,1,1.90,0.0,1.90
1,1,014_00817_3_81_20190417_14:57:00,14595593,94011,62002,20190417,20190303,817,14,1,1.53,0.0,1.53
2,2,014_00817_3_67_20190927_12:51:00,14595593,94011,62002,20190927,20190902,817,14,1,2.03,0.0,2.03
3,3,014_00380_7_68_20190315_13:18:00,238487018,94011,62002,20190315,20190202,380,14,1,2.02,0.0,2.02
4,4,014_00811_525_68_20200131_11:53:00,300620568,94011,62002,20200131,20191304,811,14,1,1.96,0.0,1.96


### Combine the transactions with products

In [57]:
tlog = pd.merge(tlog, product[["upc_no","upc_desc"]], on='upc_no', how='inner')
tlog

,Unnamed: 0,transaction_id,customer_id,upc_no,category_no_level_1,trans_dt,week_no,store_id,region_cd,purchase_unit,sales_amt,disc_amt,purchase_price,upc_desc
0,0,014_00817_4_121_20190313_18:04:00,14595593,94011,62002,20190313,20190202,817,14,1,1.90,0.00,1.9000,DOLE BANANAS TAPED ORGNC
1,1,014_00817_3_81_20190417_14:57:00,14595593,94011,62002,20190417,20190303,817,14,1,1.53,0.00,1.5300,DOLE BANANAS TAPED ORGNC
2,2,014_00817_3_67_20190927_12:51:00,14595593,94011,62002,20190927,20190902,817,14,1,2.03,0.00,2.0300,DOLE BANANAS TAPED ORGNC
3,3,014_00380_7_68_20190315_13:18:00,238487018,94011,62002,20190315,20190202,380,14,1,2.02,0.00,2.0200,DOLE BANANAS TAPED ORGNC
4,4,014_00811_525_68_20200131_11:53:00,300620568,94011,62002,20200131,20191304,811,14,1,1.96,0.00,1.9600,DOLE BANANAS TAPED ORGNC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595301,595301,014_00466_17_131_20190630_13:39:00,330734391,20799300000,52201,20190630,20190602,466,14,1,20.22,0.00,20.2200,PK BUTT SR 1PC COV
595302,595302,014_00466_10_159_20191006_20:06:00,330734391,20799300000,52201,20191006,20190904,466,14,2,38.09,0.00,19.0450,PK BUTT SR 1PC COV
595303,595303,014_00448_503_43_20190605_08:57:00,116192279,20799300000,52201,20190605,20190502,448,14,1,11.96,6.43,11.9600,PK BUTT SR 1PC COV
595304,595304,014_00355_510_82_20190821_13:54:00,272781162,20799300000,52201,20190821,20190801,355,14,1,9.66,0.00,9.6600,PK BUTT SR 1PC COV


### Cleaning date column

In [58]:
for i in range(len(tlog["week_no"])):
    tlog["week_no"][i] = str(tlog["week_no"][i])#[:4]+"-"+tlog["week_no"].astype(str)[4:6]+"-" + tlog["week_no"][i].astype(str)[6:8]

C:\Users\rtd91\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [59]:
tlog["week_no"] = tlog["week_no"].apply(lambda x:str(x))
tlog["week_no"] = tlog["week_no"].apply(lambda x: x[:4]+"-"+x[4:6]+"-"+x[6:8])

In [60]:
tlog["week_no"] = tlog["week_no"].apply(lambda x: datetime.strptime(x,"%Y-%d-%m"))

In [61]:
results = tlog.groupby(["customer_id","week_no"])["upc_desc"].apply(list)

In [18]:
len(results)

38391

In [62]:
res1= pd.DataFrame(results)

In [14]:
res1

upc_desc
customer_id week_no                                                      
2715888     2019-04-04  [SPKLN ICE CHERRY LIMEADE, KRO APPLES HNYCRISP...
7420205     2019-01-01  [SNYDR FF MINI PRETZELS, SNYDR NBLRS SOURDOUGH...
            2019-01-02  [HTGF CHK ENHC BRST BLSL, LETTUCE BOSTON RPC, ...
            2019-01-04  [PHLLY CREAM CHS BAR 2CT, BROCCOLI CROWNS, PRS...
            2019-01-05  [DRIS STRAWBERRIES PREM, HEINZ TOMATO KETCHUP,...
...                                                                   ...
1881300090  2019-04-13                        [HRSHY MILK CHOCOLATE BARS]
1975400030  2019-02-09                           [KROGER APPLE CIDER NFC]
            2019-02-10  [NSTL SPLSH WILD BERRY 6PK, NSTL SPLSH LEMON 6...
            2019-03-05              [GATORADE FRCE GRAPE, KRO 2% RF MILK]
            2019-03-07  [GATORADE FRCE GRAPE, KRO KICK OF LIME TORT CH...

[38391 rows x 1 columns]

### Cleaning the dataframe according to our requirements

In [63]:
customer_id=[]
dates=[]
for i in range(len(res1.index)):
    customer_id.append(res1.index[i][0])
    dates.append(res1.index[i][1])

In [64]:
upc_desc = list(res1["upc_desc"])

In [112]:
df = pd.DataFrame({"customer_id":customer_id, "dates":dates, "upc_desc":upc_desc})

In [113]:
df["num_products"] = df["upc_desc"].apply(lambda x: len(x))

In [114]:
df["flag"] = 0
for i in range(len(df)):
    if ('SPKLN ICE CHERRY LIMEADE' in df["upc_desc"][i] and 'SPKLN ICE PEACH NECTARINE' in df["upc_desc"][i]):
        df["flag"][i] = 1    
    

C:\Users\rtd91\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [105]:
test = df[df["flag"]==1]

In [106]:
df

,customer_id,dates,upc_desc,num_products,flag
0,2715888,2019-04-04,"[SPKLN ICE CHERRY LIMEADE, KRO APPLES HNYCRISP...",4,1
1,7420205,2019-01-01,"[SNYDR FF MINI PRETZELS, SNYDR NBLRS SOURDOUGH...",4,0
2,7420205,2019-01-02,"[HTGF CHK ENHC BRST BLSL, LETTUCE BOSTON RPC, ...",26,0
3,7420205,2019-01-04,"[PHLLY CREAM CHS BAR 2CT, BROCCOLI CROWNS, PRS...",21,0
4,7420205,2019-01-05,"[DRIS STRAWBERRIES PREM, HEINZ TOMATO KETCHUP,...",30,0
...,...,...,...,...,...
38386,1881300090,2019-04-13,[HRSHY MILK CHOCOLATE BARS],1,0
38387,1975400030,2019-02-09,[KROGER APPLE CIDER NFC],1,0
38388,1975400030,2019-02-10,"[NSTL SPLSH WILD BERRY 6PK, NSTL SPLSH LEMON 6...",3,0
38389,1975400030,2019-03-05,"[GATORADE FRCE GRAPE, KRO 2% RF MILK]",2,0


In [115]:
#df.to_csv("C:/Users/rtd91/code/ml-deployment-coe/projects/embedding_example/data/test.csv")

In [93]:
df = pd.read_csv("C:/Users/rtd91/code/ml-deployment-coe/projects/embedding_example/data/test.csv")

In [73]:
"""def test_model(df):
    max_scores = [] #scores is the number of matches
    products_similarity = {} #customer:scores # inter customer with same customer #intra customer with another customer
    transaction_id=[]
    indice_max_val = []
    index1 = []
    for i in range(len(df)):
        index1.append(i)
        scores = []
        index2 = []
        for j in range(len(df)):
            ## This condition is so that the record of customer is not compared to the same record
            if i == j:
                j=j+1
                continue
            intra_match = 0
            inter_match = 0
                
            #print((set(df["upc_desc"][i]).intersection(set(df["upc_desc"][j]))))
            transaction_id.append(str(i)+"&"+"")
            scores.append((len(set(df["upc_desc"][i]).intersection(set(df["upc_desc"][j])))))
            index2.append(j)
        max_scores.append(max(scores))
        indice_max_val.append(scores.index(max_scores[i]))
        break
    return max_scores, index1, indice_max_val, set(df["upc_desc"][index1][0]) ^ set(df["upc_desc"][indice_max_val][0]) #returns non-intersection items"""

In [78]:
"""test_model(df.head(5))"""

([0], [0], [0], set())

## Converting the column according to model input requirements

In [119]:
df["upc_desc"] = df["upc_desc"].apply(lambda x: ','.join(x))

In [111]:
df["upc_desc"][0]

'SPKLN ICE CHERRY LIMEADE,KRO APPLES HNYCRISP POUCH,SPKLN ICE BLACK CHERRY,SPKLN ICE PEACH NECTARINE'

## Dividing the data into train and validation datasets

In [120]:
customers = df["customer_id"].unique().tolist()
random.shuffle(customers)

# extract 90% of customer ID's
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]

# split data into train and validation set
train_df = df[df['customer_id'].isin(customers_train)]
validation_df = df[~df['customer_id'].isin(customers_train)]

In [121]:
# list to capture purchase history of the customers
purchases_train = []

# populate the list with the product codes
for i in tqdm(customers_train):
    temp = train_df[train_df["customer_id"] == i]["upc_desc"].tolist()
    purchases_train.append(temp)

100%|████████████████████████████████████████████████████████████████████████████| 1119/1119 [00:00<00:00, 1202.90it/s]


In [122]:
# list to capture purchase history of the customers
purchases_val = []

# populate the list with the product codes
for i in tqdm(validation_df['customer_id'].unique()):
    temp = validation_df[validation_df["customer_id"] == i]["upc_desc"].tolist()
    purchases_val.append(temp)

100%|██████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 1516.25it/s]


In [125]:
##Make the model more memory efficient
model.init_sims(replace=True)

C:\Users\rtd91\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  


In [126]:
print(model)

Word2Vec(vocab=69, vector_size=100, alpha=0.03)


In [127]:
words = list(model.wv.index_to_key)

In [131]:
words[1]

'DOLE BANANAS'

%md
## We can just see 69 words here since the default min_count of gensim word2Vec is equal to 5 so only words occuring greater than or equal to 5 times are added in the vocab.
Reference: https://stackoverflow.com/questions/56033651/words-missing-from-trained-word2vec-model-vocabulary

In [160]:
words

['KRO 2% RF MILK',
 'DOLE BANANAS',
 'DELI SALAD BAR',
 'KRO PURIFIED WATER 32PK',
 'KRO VITAMIN D WHOLE MILK',
 'BUD LIGHT 24PK CAN',
 'MTN DEW',
 'KRO GRD A LG EGGS RACK',
 'STO 2%RF MILK',
 'KRO PURIFIED WATER 24PK',
 'KRO GRD A LRG EGGS RCK',
 'WHITE CLAW 12 PK CAN',
 'HMCTY ICE NUGGETS 7LB',
 'COKE CLASSIC CONTOUR BTL',
 'ICEBERG LETTUCE',
 'BH BQT MINUET',
 'MONSTER ENERGY',
 'DELI CKN WHL RSTD BIRD H',
 'BKRY SS DONUTS',
 'DOLE BANANAS TAPED ORGNC',
 'KRO 1% LF MILK',
 'SAS GRAN SUGAR PLLT',
 'KRO DISTILLED WATER',
 'KRO 30 DR BTH TSS',
 'DIET COKE 12PK',
 'KRO 2%RF MILK',
 'BH BQT HARMONY',
 'ICE CRUSHED CUBED',
 'KRO CIN REDS WATER',
 'RED BULL ENERGY DRINK',
 'KALE BUNCH RPC',
 'POST HNY BNCH OAT W/ALMND',
 'DERPRK SPRING WATER 24PK',
 'BHAUS BQT DUET',
 'DASANI PURE WATER 24PK',
 'COKE ZERO',
 'MILLER LITE 24PK CAN',
 'KRO 1%LF MILK',
 'KALE BUNCH RPC,KALE BUNCH RPC',
 'SNCKRS BAR',
 'KRO POTATOES IDAHO PREM',
 'PICK SIX 6PK BTL',
 'WRGLY EXTRA SPEARMINT GUM',
 'DIET MTN DEW

In [135]:
products = train_df[["upc_desc"]]

# remove duplicates
products.drop_duplicates(inplace=True, subset='upc_desc', keep="last")

# create product-ID and product-description dictionary
#products_dict = products.groupby('upc_desc')['Description'].apply(list).to_dict()

C:\Users\rtd91\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [164]:
def test_model(customer_id=190462787):
    df_cust = df[df["customer_id"]==customer_id] ## for multiple customer , using .isin()
    df_cust_prod = df_cust.groupby('customer_id')["upc_desc"].apply(list)
    recommended_products = []
    ## Can add another outer for loop for multiple customer recommendation
    ## Finding the intersection of above 69 vocab products and the customer's bought products
    most_common_bought_products = set(words).intersection(set(df_cust_prod[customer_id]))
    for products in most_common_bought_products:
        recommended_products.append(model.wv.most_similar(products, topn=3))
    
    ## add 2 more conditions:
    ## first make recommended products unique and convert it into a set
    ## second check if recommended products already exists in df_cust_prod[customer_id])
    return recommended_products

In [167]:
test_model(190462787)

[[('KALE BUNCH RPC', 0.357295960187912),
  ('POST HNY BNCH OAT W/ALMND', 0.35685306787490845),
  ('DERPRK SPRING WATER 24PK', 0.33967921137809753)]]

### For customer having no recommendations, we can use customer2vec model

In [171]:
for cust_id in customers:
    print(cust_id)
    print(test_model(cust_id))

190462787
[[('KALE BUNCH RPC', 0.357295960187912), ('POST HNY BNCH OAT W/ALMND', 0.35685306787490845), ('DERPRK SPRING WATER 24PK', 0.33967921137809753)]]
146867350
[]
299207048
[]
92267703
[]
308559012
[]
259401963
[[('KRO PURIFIED WATER 32PK', 0.34293630719184875), ('KALE BUNCH RPC', 0.3079712688922882), ('DASANI PURE WATER 24PK', 0.3035476803779602)], [('KITKAT CRISP WAFER', 0.18309709429740906), ('DELI CKN FR SLF SRV 8P H', 0.1500629186630249), ('MILLER LITE 24PK CAN', 0.14767128229141235)]]
101025127
[[('KALE BUNCH RPC,KALE BUNCH RPC', 0.6573269367218018), ('KALE BUNCH RPC', 0.6572238206863403), ('STO 2%RF MILK', 0.6259320974349976)]]
209964619
[]
101913708
[]
128062453
[]
110982127
[]
147488885
[]
299535125
[]
338169082
[[('BUD LIGHT 24PK CAN', 0.4054492712020874), ('KALE BUNCH RPC', 0.3748836815357208), ('STO 2%RF MILK', 0.3745972514152527)], [('KRO PURIFIED WATER 24PK', 0.3870144486427307), ('DASANI PURE WATER 24PK', 0.3767983019351959), ('KALE BUNCH RPC', 0.3621307611465454)]]

[]
134503513
[]
216038677
[]
100033371
[]
107555554
[]
242227100
[]
77252100
[]
290761603
[[('KRO 2% RF MILK', 0.21689674258232117), ('KRO 1% LF MILK', 0.21043917536735535), ('COKE CLASSIC 12PK', 0.20376767218112946)], [('BUD LIGHT 24PK CAN', 0.36111050844192505), ('KRO PURIFIED WATER 32PK', 0.3399185538291931), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.33530083298683167)]]
98671672
[]
147786932
[]
108857000
[]
12666723
[]
223791875
[]
329473366
[[('KALE BUNCH RPC', 0.8698782920837402), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.8689242601394653), ('STO 2%RF MILK', 0.812996506690979)]]
102019178
[]
148181429
[]
269310107
[]
233592881
[]
147723099
[]
127017317
[]
100315036
[[('HMCTY ICE NUGGETS 22LB', 0.22136758267879486), ('PEPSI', 0.1876983493566513), ('BKRY SS DONUTS', 0.1681305170059204)]]
147871648
[]
102279589
[]
334639058
[]
146913527
[]
325962500
[]
299161612
[]
186220133
[[('KALE BUNCH RPC', 0.46025264263153076), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.4376057982444763), ('STO 2%RF MILK', 0.436

[]
341202491
[[('ICEBERG LETTUCE', 0.2603684663772583), ('DELI CKN FR SLF SRV 8P H', 0.25517693161964417), ('KRO 1%LF MILK', 0.21925628185272217)]]
183661033
[[('BUD LIGHT 24PK CAN', 0.4942016303539276), ('KALE BUNCH RPC', 0.4596514105796814), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.45333313941955566)]]
244935154
[[('KRO HEAVY WHIPPING CREAM', 0.25994011759757996), ('BUD LIGHT 24PK CAN', 0.2513977885246277), ('DIET COKE 6PK', 0.21376897394657135)]]
92666424
[]
87470635
[]
342350374
[[('KALE BUNCH RPC', 0.45346736907958984), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.447645902633667), ('DERPRK SPRING WATER 24PK', 0.423600435256958)]]
147681015
[[('KALE BUNCH RPC,KALE BUNCH RPC', 0.5202544927597046), ('KALE BUNCH RPC', 0.492653489112854), ('BUD LIGHT 24PK CAN', 0.476080983877182)]]
337788637
[[('KALE BUNCH RPC,KALE BUNCH RPC', 0.6573269367218018), ('KALE BUNCH RPC', 0.6572238206863403), ('STO 2%RF MILK', 0.6259320974349976)], [('STO 2%RF MILK', 0.439344584941864), ('KALE BUNCH RPC', 0.4346727430820

195050879
[[('KALE BUNCH RPC', 0.45346736907958984), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.447645902633667), ('DERPRK SPRING WATER 24PK', 0.423600435256958)], [('DIET COKE 12PK', 0.2996777892112732), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.2644175589084625), ('STO 2%RF MILK', 0.26067110896110535)]]
98548040
[[('DIET COKE 12PK', 0.2996777892112732), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.2644175589084625), ('STO 2%RF MILK', 0.26067110896110535)]]
252264080
[[('KALE BUNCH RPC,KALE BUNCH RPC', 0.4599861204624176), ('KALE BUNCH RPC', 0.445765882730484), ('STO 2%RF MILK', 0.4221833348274231)], [('DIET COKE 12PK', 0.48909515142440796), ('BUD LIGHT 24PK CAN', 0.4245010018348694), ('KALE BUNCH RPC', 0.4238097667694092)]]
210978506
[]
12777498
[]
344210265
[[('STO 2%RF MILK', 0.439344584941864), ('KALE BUNCH RPC', 0.4346727430820465), ('CTO BATH TISSUE', 0.4201456606388092)]]
103797658
[]
262460750
[]
243415805
[]
329473860
[[('KALE BUNCH RPC,KALE BUNCH RPC', 0.4988705515861511), ('BUD LIGHT 24PK CAN', 

[]
343115385
[[('KRO PURIFIED WATER 32PK', 0.34293630719184875), ('KALE BUNCH RPC', 0.3079712688922882), ('DASANI PURE WATER 24PK', 0.3035476803779602)]]
149034788
[]
98618264
[]
104596323
[]
84969512
[]
315264907
[[('MILLER LITE 24PK CAN', 0.2669658064842224), ('KRO PURIFIED WATER 24PK', 0.26204556226730347), ('KITKAT CRISP WAFER', 0.2526586949825287)]]
204012242
[]
270803792
[]
158720993
[]
147928719
[]
16534545
[]
148163403
[[('STO 2%RF MILK', 0.439344584941864), ('KALE BUNCH RPC', 0.4346727430820465), ('CTO BATH TISSUE', 0.4201456606388092)]]
147762524
[]
329906174
[]
149304669
[[('COKE CLASSIC', 0.3199152648448944), ('STO 2%RF MILK', 0.31376686692237854), ('SAS GRAN SUGAR PLLT', 0.30172204971313477)], [('KALE BUNCH RPC', 0.46025264263153076), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.4376057982444763), ('STO 2%RF MILK', 0.4360925257205963)]]
228994098
[]
320144137
[]
118180688
[]
216527574
[]
89615672
[[('STO 2%RF MILK', 0.4248989224433899), ('KALE BUNCH RPC', 0.4227912724018097), ('KAL

[]
272746719
[[('KRO HEAVY WHIPPING CREAM', 0.25994011759757996), ('BUD LIGHT 24PK CAN', 0.2513977885246277), ('DIET COKE 6PK', 0.21376897394657135)]]
330763330
[]
238450347
[]
343486796
[]
100451536
[[('KALE BUNCH RPC,KALE BUNCH RPC', 0.5202544927597046), ('KALE BUNCH RPC', 0.492653489112854), ('BUD LIGHT 24PK CAN', 0.476080983877182)], [('COKE CLASSIC', 0.4890950918197632), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.4687293469905853), ('POST HNY BNCH OAT W/ALMND', 0.4420761466026306)], [('KALE BUNCH RPC,KALE BUNCH RPC', 0.6573269367218018), ('KALE BUNCH RPC', 0.6572238206863403), ('STO 2%RF MILK', 0.6259320974349976)]]
146650595
[]
243415703
[]
94014455
[]
147306032
[]
86894937
[]
259743284
[[('KALE BUNCH RPC', 0.46195340156555176), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.45312488079071045), ('STO 2%RF MILK', 0.4478839635848999)]]
299535333
[]
148504310
[]
116321462
[]
148225012
[]
225887738
[]
147136435
[[('KALE BUNCH RPC', 0.45346736907958984), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.447645902633

[[('BUD LIGHT 24PK CAN', 0.4942016303539276), ('KALE BUNCH RPC', 0.4596514105796814), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.45333313941955566)]]
100295404
[]
324634270
[[('KRO PURIFIED WATER 24PK', 0.3870144486427307), ('DASANI PURE WATER 24PK', 0.3767983019351959), ('KALE BUNCH RPC', 0.3621307611465454)], [('KALE BUNCH RPC', 0.4285537898540497), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.3826340436935425), ('STO 2%RF MILK', 0.35212093591690063)]]
132737705
[]
225625321
[]
147389138
[]
202792488
[]
90307107
[]
263782916
[[('KRO HEAVY WHIPPING CREAM', 0.25994011759757996), ('BUD LIGHT 24PK CAN', 0.2513977885246277), ('DIET COKE 6PK', 0.21376897394657135)], [('KALE BUNCH RPC', 0.33693283796310425), ('DOLE BANANAS', 0.3346328139305115), ('DELI SALAD BAR', 0.3334997594356537)]]
148334983
[]
86681430
[]
253177948
[]
148017146
[]
93125458
[]
108336876
[]
290415347
[]
224335638
[]
147251692
[[('KALE BUNCH RPC,KALE BUNCH RPC', 0.4988705515861511), ('BUD LIGHT 24PK CAN', 0.4734945595264435), ('STO 2%RF 

[]
148755492
[]
212312304
[]
15321804
[]
218848973
[]
96912368
[]
264472840
[]
116318566
[[('KALE BUNCH RPC,KALE BUNCH RPC', 0.5202544927597046), ('KALE BUNCH RPC', 0.492653489112854), ('BUD LIGHT 24PK CAN', 0.476080983877182)]]
146794249
[]
147267467
[[('KALE BUNCH RPC,KALE BUNCH RPC', 0.6573269367218018), ('KALE BUNCH RPC', 0.6572238206863403), ('STO 2%RF MILK', 0.6259320974349976)]]
17732472
[]
336636922
[]
146717528
[]
146630041
[[('KRO HEAVY WHIPPING CREAM', 0.25994011759757996), ('BUD LIGHT 24PK CAN', 0.2513977885246277), ('DIET COKE 6PK', 0.21376897394657135)]]
146750875
[]
320274410
[]
147152179
[]
86537706
[]
108378653
[[('KALE BUNCH RPC', 0.2934151291847229), ('STO 2%RF MILK', 0.28568035364151), ('HMCTY ICE NUGGETS 22LB', 0.2849181890487671)]]
98776372
[]
336614428
[]
146626653
[]
185559607
[[('STO 2%RF MILK', 0.439344584941864), ('KALE BUNCH RPC', 0.4346727430820465), ('CTO BATH TISSUE', 0.4201456606388092)]]
184571359
[]
326396085
[]
345399021
[]
98786005
[]
242866054
[]
33

[]
147332472
[]
345846428
[]
102059055
[]
101870779
[[('COKE CLASSIC', 0.4890950918197632), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.4687293469905853), ('POST HNY BNCH OAT W/ALMND', 0.4420761466026306)]]
111302398
[]
332802827
[[('DIET COKE 12PK', 0.20309126377105713), ('BH BQT HARMONY', 0.17082688212394714), ('KRO VITAMIN D WHOLE MILK', 0.16518577933311462)]]
94147305
[]
89591315
[[('ICEBERG LETTUCE', 0.2603684663772583), ('DELI CKN FR SLF SRV 8P H', 0.25517693161964417), ('KRO 1%LF MILK', 0.21925628185272217)]]
158086855
[[('BUD LIGHT 24PK CAN', 0.5987051725387573), ('KALE BUNCH RPC', 0.5926470160484314), ('KALE BUNCH RPC,KALE BUNCH RPC', 0.5899031162261963)]]
314197711
[]
312694501
[]
15504528
[]
94943371
[]
192136490
[]
149108569
[]
129961090
[[('KALE BUNCH RPC', 0.33693283796310425), ('DOLE BANANAS', 0.3346328139305115), ('DELI SALAD BAR', 0.3334997594356537)]]
98664220
[]
101835823
[]
147005423
[]
106624150
[]
132918430
[]
264114235
[[('MILLER LITE 24PK CAN', 0.2669658064842224), ('KR